# Zagadki

![image-2.png](attachment:image-2.png)

## Wstęp
Zagadki od wieków fascynują ludzi, pobudzając ich umysły do kreatywnego i logicznego myślenia. Od prostych łamigłówek po głębokie zagadki filozoficzne, stanowią one nie tylko formę rozrywki, ale również sztukę rozumienia języka i logicznego wnioskowania. W tym zadaniu będziesz rozwiązywał/a zagadki, polegające na odgadnięciu słowa na podstawie opisu. Wszystkie zagadki wymyślił ChatGPT (ale nie powiemy, która dokładnie wersja i w jaki sposób zachęcona), stąd niektóre mogą być trochę dziwne... Szacujemy, że ludzie są w stanie rozwiązać poprawnie trochę ponad 60% z nich. A jak dobry będzie Twój program?

## Zadanie
Napisz funckję `answer_riddle` która rozwiąże podaną na wejściu zagadkę. Rozwiązaniem jest zawsze jedno słowo. Przykładowe zagadki:

- **zagadka:** kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem <br>
  **odpowiedź:** pasażerka
- **zagadka:** emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu<br>
  **odpowiedź:** miłość


Naszym kryterium będzie `odwrócona średnia harmoniczna` ([Mean Reciprocal Rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)), która działa w następujący sposób: <br>
Jeżeli na zwróconej przez Twoją funkcję liście znajdzie się prawidłowa odpowiedź, otrzymasz wówczas punkty: dokładnie $\frac{1}{k}$ punktów, gdzie $k$ jest pozycją słowa na liście. W szczególności, jeżeli Twój program zgadnie słowo (czyli umieści je na pierwszej pozycji), to otrzymasz 1 punkt. Ostatecznym kryterium jest uśredniona liczba punktów ze wszystkich zagadek.

Powyższe kryterium jest zaimplementowane poniżej przez nas.

## Ograniczenia
- Twoje finalne rozwiązanie będzie testowane w środowisku **bez** GPU.
- Twoja funkcja powinna działać na tyle szybko, aby program był w stanie udzielić odpowiedzi na 100 zagadek w maksymalnie 2 minuty bez użycia GPU.

## Dane
Dane dostępne dla Ciebie w tym zadaniu to:
* `zagadki_do_testow_clean.txt` - około 2000 przykładowych zagadek
* `plwiktionary_definitions_clean.txt` -  plik z definicjami słów wziętymi z [pl.wiktionary.org](https://pl.wiktionary.org/wiki/pl). Z wszystkich definicji z pl.wiktionary.org wzięliśmy definicje 8094 najbardziej popularnych rzeczowników (częstości liczone zgodnie z korpusem https://2018.poleval.pl/index.php/tasks#task3). Uwaga: poprawne rozwiązanie każdej zagadki znajduje się w tym pliku!

* `superbazy_clean.txt` - formy bazowe polskich słów, przygotowane na podstawie projektu https://github.com/morfologik/polimorfologik

* Wektory osadzeń słów bazowych, wytrenowane modelem Word2Vec z biblioteki Gensim, na korpusie PolEval 2018 Task3

## Uwagi i wskazówki
- Dla każdej zagadki, Twoja funkcja powinna zwrócić listę słów (co najwyżej 20), w kolejności od najbardziej (wg Twojego programu) prawdopodobnej odpowiedzi na zagadkę, do najmniej.
- Twoje rozwiazanie bedzie testowane bez dostepu do internetu

## Pliki Zgłoszeniowe
Tylko ten notebook.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` będziesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

# Kod startowy

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
import nltk
from nltk.tokenize import word_tokenize as tokenize
from collections import defaultdict as dd
import math
from gensim.models import Word2Vec
import gdown
import random
import os
from tqdm import tqdm

## Ładowanie danych

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
path_to_data = 'data/'

bases = {}
# Dictionary mapping words to their base words
all_word_definitions = dd(list)
# Dictionary containing all base words inverse document frequency
base_idf = dd(int)

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def get_word_base(word):
    global bases
    word = word.lower()
    ret = bases.get(word)

    if ret:
        ret = ret.split("_")
        return ret[0]
        # return ret
    return word

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    if not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.syn1neg.npy") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.wv.vectors.npy"):
            gdown.download_folder(url="https://drive.google.com/drive/folders/1P72og_ORfL3Ojf27n-g06DT0ENduPy8C?usp=sharing", output=f"./{path_to_data}")
    nltk.download('punkt')

Retrieving folder contents


Processing file 1dCYKzdg6TihyckNM9Zxr1g9lWT1BzDXI plwiktionary_definitions_clean.txt
Processing file 1q6Ki5Y66gugM30oFcCtJj7ocMJymskSk superbazy_clean.txt
Processing file 1GZPNIR16bxFnzvbIYDLacn8TcKwiGnIh w2v_polish_lemmas.model
Processing file 1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC w2v_polish_lemmas.model.syn1neg.npy
Processing file 1RY0Ftfx_-nPUbddYCvHq9p8yCcXbUrYS w2v_polish_lemmas.model.wv.vectors.npy
Processing file 18wrF9VyESTvIT9Z_TqUd_2wV4lP2As3t zagadki_do_testow_clean.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1dCYKzdg6TihyckNM9Zxr1g9lWT1BzDXI
To: /content/data/zagadki/plwiktionary_definitions_clean.txt
100%|██████████| 1.59M/1.59M [00:00<00:00, 47.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1q6Ki5Y66gugM30oFcCtJj7ocMJymskSk
To: /content/data/zagadki/superbazy_clean.txt
100%|██████████| 43.7M/43.7M [00:01<00:00, 35.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GZPNIR16bxFnzvbIYDLacn8TcKwiGnIh
To: /content/data/zagadki/w2v_polish_lemmas.model
100%|██████████| 40.8M/40.8M [00:01<00:00, 33.5MB/s]


FileURLRetrievalError: Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC

but Gdown can't. Please check connections and permissions.

In [ ]:
files = os.listdir(path_to_data + "zagadki")
print("Pobrane pliki:")
for file in files:
    print(file)



In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/superbazy_clean.txt'):
    word,base = x.lower().split()
    bases[word] = base

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
model = Word2Vec.load(f'{path_to_data}/zagadki/w2v_polish_lemmas.model')

In [ ]:
categories = {
    '{{techn}}': 'technologia',
    '{{B}}': 'bieżący',
    '{{zootechn}}': 'zootechnika',
    '{{stpol}}': 'studia polityczne',
    '{{więz}}': 'więzienie',
    '{{kartogr}}': 'kartografia',
    '{{przym}}': 'przymiotnik',
    '{{ekon}}': 'ekonomia',
    '{{mitgerm}}': 'mitologia germańska',
    '{{amer}}': 'Ameryka',
    '{{sf}}': 'science fiction',
    '{{kujawy}}': 'Kujawy',
    '{{ż}}': 'życie',
    '{{przestęp}}': 'przestępstwo',
    '{{miner}}': 'minerał',
    '{{księg}}': 'księgowość',
    '{{nłac}}': 'najwyższa izba kontroli',
    '{{psych}}': 'psychologia',
    '{{nawig}}': 'nawigacja',
    '{{fonet}}': 'fonetyka',
    '{{dypl}}': 'dyplomacja',
    '{{fakt}}': 'faktor',
    '{{zdrobn}}': 'zdrobnienie',
    '{{hotel}}': 'hotel',
    '{{praw}}': 'prawo',
    '{{mitgr}}': 'mitologia grecka',
    '{{plast}}': 'plastyka',
    '{{syst}}': 'system',
    '{{monet}}': 'moneta',
    '{{muz}}': 'muzyka',
    '{{socjol}}': 'socjologia',
    '{{itp}}': 'i tym podobne',
    '{{zool}}': 'zoologia',
    '{{zgrub}}': 'zgrubienie',
    '{{paleont}}': 'paleontologia',
    '{{pot}}': 'potencjał',
    '{{mit}}': 'mitologia',
    '{{speleol}}': 'speleologia',
    '{{chem}}': 'chemia',
    '{{grub}}': 'grubiański',
    '{{n}}': 'nazwa',
    '{{kult}}': 'kultura',
    '{{metrol}}': 'metrologia',
    '{{szkocang}}': 'szkoła czarodziejów angielskich',
    '{{astronaut}}': 'astronautyka',
    '{{bot}}': 'botanika',
    '{{łow}}': 'łowiectwo',
    '{{urb}}': 'urbanistyka',
    '{{filoz}}': 'filozofia',
    '{{tosk}}': 'toskania',
    '{{zarz}}': 'zarządzanie',
    '{{płn}}': 'północ',
    '{{ezot}}': 'ezoteryka',
    '{{mitrzym}}': 'mitologia rzymska',
    '{{poet}}': 'poeta',
    '{{dendr}}': 'dendrologia',
    '{{odczas}}': 'od czasu',
    '{{iron}}': 'ironia',
    '{{reg}}': 'regulamin',
    '{{mar}}': 'marchew',
    '{{kulin}}': 'kulinaria',
    '{{mitsłow}}': 'mitologia słowiańska',
    '{{fot}}': 'fotografia',
    '{{żegl}}': 'żeglarstwo',
    '{{miar}}': 'miara',
    '{{fż}}': 'feminizm',
    '{{żyd}}': 'żyd',
    '{{hutn}}': 'hutnictwo',
    '{{przesz}}': 'przeszłość',
    '{{kib}}': 'kibic',
    '{{myśl}}': 'myślenie',
    '{{pejor}}': 'pejoratywny',
    '{{ogrod}}': 'ogród',
    '{{itd}}': 'i tak dalej',
    '{{dial}}': 'dialektologia',
    '{{jeźdz}}': 'jeździectwo',
    '{{flis}}': 'flisak',
    '{{zobacz}}': 'zobacz',
    '{{gwara}}': 'gwara',
    '{{rel}}': 'religia',
    '{{herp}}': 'herpetologia',
    '{{adm}}': 'administracja',
    '{{icht}}': 'ichtiologia',
    '{{także}}': 'także',
    '{{telew}}': 'telewizja',
    '{{m}}': 'mężczyzna',
    '{{gin}}': 'gin',
    '{{pogard}}': 'pogarda',
    '{{odl}}': 'odległość',
    '{{jubil}}': 'jubiler',
    '{{biur}}': 'biuro',
    '{{ryb}}': 'rybak',
    '{{obraź}}': 'obraza',
    '{{rzem}}': 'rzemieślnik',
    '{{hig}}': 'higiena',
    '{{spoż}}': 'spożywczy',
    '{{harc}}': 'harcerski',
    '{{peryfr}}': 'peryfrazowany',
    '{{fryzj}}': 'fryzjerstwo',
    '{{środ}}': 'środek',
    '{{bryt}}': 'brytyjski',
    '{{liter}}': 'literatura',
    '{{jęz}}': 'język',
    '{{krym}}': 'kryminalistyka',
    '{{ts}}': 'ts',
    '{{poznań}}': 'Poznań',
    '{{szach}}': 'szachy',
    '{{wikipedia}}': 'Wikipedia',
    '{{herald}}': 'heraldyka',
    '{{kynol}}': 'kynologia',
    '{{gastr}}': 'gastronomia',
    '{{bud}}': 'budownictwo',
    '{{zaw}}': 'zawód',
    '{{log}}': 'logika',
    '{{stud}}': 'studium',
    '{{kraków}}': 'Kraków',
    '{{fant}}': 'fantastyka',
    '{{austral}}': 'australia',
    '{{ręk}}': 'ręka',
    '{{żart}}': 'żart',
    '{{infant}}': 'infantylizm',
    '{{meteorol}}': 'meteorologia',
    '{{typogr}}': 'typografia',
    '{{kośc}}': 'kościół',
    '{{uczn}}': 'uczeń',
    '{{turyst}}': 'turystyka',
    '{{geol}}': 'geologia',
    '{{publ}}': 'publikacja',
    '{{antrop}}': 'antropologia',
    '{{łódź}}':'łódz',
    '{{kraw}}': 'krawiectwo',
    '{{hist}}': 'historia',
    '{{symbol}}': 'symbol',
    '{{geod}}': 'geodezja',
    '{{stomat}}': 'stomatologia',
    '{{geofiz}}': 'geofizyka',
    '{{starop}}': 'staropolski',
    '{{biol}}': 'biologia',
    '{{polit}}': 'polityka',
    '{{choreogr}}': 'choreografia',
    '{{telekom}}': 'telekomunikacja',
    '{{agrot}}': 'agroturystyka',
    '{{anat}}': 'anatomia',
    '{{geogr}}': 'geografia',
    '{{os}}': 'osoba',
    '{{mitbask}}': 'mitologia baszkirska',
    '{{wędk}}': 'wędkarstwo',
    '{{roln}}': 'rolnictwo',
    '{{biochem}}': 'biochemia',
    '{{opt}}': 'optyka',
    '{{fryz}}': 'fryzura',
    '{{pedag}}': 'pedagogika',
    '{{przest}}': 'przestój',
    '{{bibliot}}': 'biblioteka',
    '{{mors}}': 'morski',
    '{{daw}}': 'dawny',
    '{{eduk}}': 'edukacja',
    '{{slang}}': 'slang',
    '{{warmia}}': 'Warmia',
    '{{wet}}': 'weterynaria',
    '{{kolej}}': 'kolej',
    '{{lit}}': 'literatura',
    '{{kosmet}}': 'kosmetyka',
    '{{poligr}}': 'poligrafia',
    '{{elektr}}': 'elektryka',
    '{{hand}}': 'handel',
    '{{wulg}}': 'wulgaryzm',
    '{{mech}}': 'mechanika',
    '{{etn}}': 'etnografia',
    '{{społ}}': 'społeczeństwo',
    '{{górn}}': 'górnik',
    '{{rub}}': 'rubin',
    '{{eufem}}': 'eufemizm',
    '{{etc}}': 'etcetera',
    '{{bibl}}': 'biblia',
    '{{ekol}}': 'ekologia',
    '{{D}}': 'dolina',
    '{{książk}}': 'książka',
    '{{wojsk}}': 'wojsko',
    '{{lotn}}': 'lotnictwo',
    '{{neol}}': 'neologizm',
    '{{szt}}': 'sztuka',
    '{{urz}}': 'urząd',
    '{{bizk}}': 'biznes',
    '{{poczt}}': 'poczta',
    '{{W}}': 'waga',
    '{{kanadang}}': 'kanadański',
    '{{bank}}': 'bank',
    '{{staroż}}': 'starożytność',
    '{{wiośl}}': 'wioślarstwo',
    '{{karc}}': 'karczma',
    '{{archit}}': 'architektura',
    '{{med}}': 'medycyna',
    '{{dosł}}': 'dosłownie',
    '{{kresy}}': 'kresy',
    '{{akust}}': 'akustyka',
    '{{leśn}}': 'leśnictwo',
    '{{przen}}': 'przemyślenie',
    '{{mikol}}': 'mikrologia',
    '{{współcz}}': 'współczesność',
    '{{posp}}': 'pospolity',
    '{{fiz}}': 'fizyka',
    '{{nauk}}': 'nauka',
    '{{ter}}': 'teren',
    '{{mot}}': 'motywacja',
    '{{astr}}': 'astronomia',
    '{{film}}': 'film',
    '{{teol}}': 'teologia',
    '{{lud}}': 'ludność',
    '{{mat}}': 'matematyka',
    '{{ofic}}': 'oficjalny',
    '{{mikrobiol}}': 'mikrobiologia',
    '{{archeol}}': 'archeologia',
    '{{Gloger}}': 'Gloger',
    '{{mebl}}': 'meblarstwo',
    '{{młodz}}': 'młodzież',
    '{{brazport}}': 'Brazylia',
    '{{M}}': 'metropolia',
    '{{pszcz}}': 'pszczelarstwo',
    '{{teatr}}': 'teatr',
    '{{pieszcz}}': 'pieszczenie',
    '{{lp}}': 'liczba porządkowa',
    '{{podn}}': 'podniecenie',
    '{{met}}': 'meteor',
    '{{gram}}': 'gramatyka',
    '{{lwów}}': 'Lwów',
    '{{stat}}': 'statystyka',
    '{{sport}}': 'sport',
    '{{hydrol}}': 'hydrolizować',
    '{{geom}}': 'geometria',
    '{{śrłac}}': 'średnio-łaciński',
    '{{białystok}}': 'Białystok',
    '{{grzecz}}': 'grzeczność',
    '{{ekspr}}': 'ekspresowy',
    '{{seks}}': 'seks',
    '{{farm}}': 'farmakologia',
    '{{rzad}}': 'rząd',
    '{{druk}}': 'druk',
    '{{fizj}}': 'fizjologia',
    '{{np}}': 'na przykład',
    '{{zwł}}': 'zwłaszcza',
    '{{filatel}}': 'filatelistyka',
    '{{blp}}': 'biologia molekularna',
    '{{folk}}': 'folklor',
    '{{włók}}': 'włókiennictwo',
    '{{przysz}}': 'przyszłość',
    '{{elektron}}': 'elektronika',
    '{{masz}}': 'maszyna',
    '{{odprzym}}': 'odprawa',
    '{{demogr}}': 'demografia',
    '{{enol}}': 'enologia',
    '{{lm}}': 'laserowa mikroobróbka',
    '{{ent}}': 'entomologia',
    '{{finans}}': 'finanse',
    '{{rad}}': 'radio',
    '{{inform}}': 'informatyka',
    '{{rzecz}}': 'rzeczownik',
    '{{gip}}': 'gips',
    '{{właśc}}': 'właściciel',
    '{{spec}}': 'specyfikacja',
    '{{numizm}}': 'numizmatyka',
    '{{ornit}}': 'ornitologia',
    '{{cuk}}': 'cukier',
    '{{lekcew}}': 'lekceważenie'
}


In [ ]:
import re

######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/plwiktionary_definitions_clean.txt'):
    word, definition = x.split('###')
    for cat in re.findall(r"{{\w+}}", definition):
        definition.replace(cat, categories[cat])
    L = word.split()
    if len(L) == 1:
        word = L[0]

        definition = set(tokenize(definition.lower()))
        all_word_definitions[word].append(definition)
        for word in set(definition):
            base_idf[get_word_base(word)] += 1


for base in base_idf:
    base_idf[base] = -math.log(base_idf[base] / len(all_word_definitions))

In [ ]:
list(set([item for sublist in categories for item in sublist]))

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
answers = []
queries = []

with open(f'{path_to_data}/zagadki/zagadki_do_testow_clean.txt') as file:
    for line in file:
        line = line.replace(';;', '').split()
        answers.append(line[0])
        queries.append(tokenize(' '.join(line[1:])))

## Kod z kryteriami oceniającymi

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def mean_reciprocal_rank(real_answers, computed_answers, K=20):
    positions = []

    for real_answer, computed_answer in zip(real_answers, computed_answers):
        if real_answer in computed_answer[:K]:
            pos = computed_answer.index(real_answer) + 1
            positions.append(1/pos)

    mrr = sum(positions) / len(real_answers)
    print ('Mean Reciprocal Rank =', mrr)

    return mrr

# Twoje rozwiązanie

To jest jedyna sekcja, w której musisz coś zrobić.

In [ ]:
def clear_riddle(riddle):
    cleared_riddle = np.array([])

    # for word in riddle:
    #     cleared_riddle = np.append(cleared_riddle, word.replace("'", " "). replace('"'," "))

    # list_to_remove = [",",".","?","-", "czy", "i", "{", "}"]
    list_to_remove = [",",".","?", "!", "#", "%", "'", "''", '"', '""',"-", "czy", "i", "{", "}", ";", "w", "lub", "(", ")", "wikipedia", "z", "do", "na", ".", ":", "_", "-","/","@", "^", "&", "*","+"]
    for el in list_to_remove:
        riddle.discard(el)

    for word in riddle:
      cleared_riddle = np.append(cleared_riddle, get_word_base(word))

    return cleared_riddle

def vectorize_sentence(riddle):
    global model

    riddle = clear_riddle(riddle)

    words = np.array([])
    weights = np.array([])

    for word in riddle:
        if (word in model.wv):
            words = np.append(words, word)
            if(word in base_idf):
                if(base_idf[word]<0):
                    weights = np.append(weights, 0)
                else:
                    weights = np.append(weights, base_idf[word])
            else:
                weights = np.append(weights, 34)

    if (len(words)>0):

        word_vectors = np.array([model.wv[word] for word in words])

        average_vector = np.average(word_vectors, axis=0, weights=weights)

        return average_vector
    return np.nan

def closest_words_to_vectorized_string(vectorized_string, topn=20):
    global model

    closest_words = []
    for word, definitions in all_word_definitions.items():
        if(get_word_base(word) in model.wv and get_word_base(word) not in riddle):
            word_vector = model.wv[get_word_base(word)]
            similarity = np.dot(vectorized_string, word_vector) / (np.linalg.norm(vectorized_string) * np.linalg.norm(word_vector))
            closest_words.append((word, similarity))
            if(word in vectorized_definitions):
                for word_vector in vectorized_definitions[word]:
                    similarity = np.dot(vectorized_string, word_vector) / (np.linalg.norm(vectorized_string) * np.linalg.norm(word_vector))
                    closest_words.append((word, similarity))

    closest_words.sort(key=lambda x: x[1], reverse=True)

    final = []
    i = 0
    while len(final) != 20:
        if(closest_words[i] not in final):
            final.append(closest_words[i])
        i += 1

    return final

In [ ]:
import numpy as np
vectorized_definitions = {}

for word, definitions in all_word_definitions.items():
            for single_definition in definitions:
                if (len(single_definition) != 0):
                    word_vector = vectorize_sentence(single_definition)
                    if (word_vector is not np.nan):
                        if (word in vectorized_definitions):
                            vectorized_definitions[word] = np.append(vectorized_definitions[word],[word_vector], axis=0)
                        else:
                            vectorized_definitions[word] = np.array([word_vector])

In [ ]:
i = 0
for word in vectorized_definitions:
    if i>10:
      break
    print(word)
    print(vectorized_definitions[word].shape)
    i+=1

In [ ]:
# ///////////////////////////////////////\
#
# PAMIĘTAJ O ZMIANIE W get_word_base()
# I o zmianie w definiton.replace("'"," ")
#
# //////////////////////////////////////

import numpy as np

def get_word_base_my(word):
    return get_word_base(word)

def answer_riddle(riddle, K):
    global model

    average_vector = vectorize_sentence(riddle)

    similar_words_with_probabilities = closest_words_to_vectorized_string(average_vector, K)

    # similar_words_with_probabilities = model.wv.similar_by_vector(average_vector, topn=K)
    similar_words = [word[0] for word in similar_words_with_probabilities]

    return similar_words

['sdf', 'sfsdf']

In [ ]:
answer_riddle(set(tokenize("kot z dżungli, dziki w paski")),20)

['dżungla',
 'żółw',
 'żółwi',
 'kasa',
 'bestia',
 'jama',
 'bestia',
 'stwór',
 'pustkowie',
 'set',
 'most',
 'monstrum',
 'potwór',
 'plon',
 'bestia',
 'puszcza',
 'małpa',
 'off',
 'monstrum',
 'bestia']

# Ewaluacja

Poniższy kod będzie służył ewaluacji rozwiązania. Po wysłaniu rozwiązania do nas zostanie wykonana funkcja `evaluate_algorithm(score_function, queries, answers, K)`, t.j. prawie identyczny kod jak niżej będzie się uruchamiał na katalogu danych `test_data` dostępnym tylko dla sprawdzających zadania.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def evaluate_algorithm(score_function, queries, answers, K):
    computed_answers = []
    i=0
    for query in tqdm(queries, desc="queries answered"):
        computed_answers.append(score_function(set(query), K=K))
        # print(queries[i])
        # print(answers[i])
        # print(computed_answers[i])
        # print("///////////////////////////////////")
        i+=1
    score = mean_reciprocal_rank(answers, computed_answers, K=K)

    return score

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    PART_OF_DATA = 100
    START = 0
    K = 20
    valid_queries = queries[START:PART_OF_DATA]
    valid_answers = answers[START:PART_OF_DATA]

    # for i in range(PART_OF_DATA):
    #   print(valid_queries[i])
    #   print(valid_answers[i])
    #   print(all_word_definitions[valid_answers[i]])

    score = evaluate_algorithm(answer_riddle, valid_queries, valid_answers, K=K)
    print(f"Score: {score}")

queries answered: 100%|██████████| 100/100 [01:38<00:00,  1.01it/s]

Mean Reciprocal Rank = 0.2344595018844244
Score: 0.2344595018844244


# ZRÓB KOPIĘ

Zrób kopięęęęęęęęęęęęęęęęę